# AI Project Sub3

In [1]:
from konlpy.tag import Kkma
import pandas as pd
import tensorflow as tf
import enum
import os
import re
from sklearn.model_selection import train_test_split
import numpy as np
from configs import DEFINES

ModuleNotFoundError: No module named 'configs'

# 데이터셋에 데이터 추가하기
## http://www.aihub.or.kr/ 에서 데이터를 가져왔다.
    - 엑셀 파일의 이름은 변경해야 한다.
    - 엑셀의 내용 중 필요한 내용만 따로 빼서 csv파일로 만들었다.
    - 길이가 너무 긴 질문이나 답은 제외 시켰다.

In [ ]:
import openpyxl
import csv

In [ ]:
# 엑셀 파일 열기
names = ['academy.xlsx', 'accommodation.xlsx', 'caffe.xlsx', 'cloth.xlsx', 'estate.xlsx', 'food.xlsx', 'play.xlsx', 'service.xlsx', 'store.xlsx']

for name in names:
    wb = openpyxl.load_workbook(name)
    # sheet 열기
    sheet = wb['Sheet1']
    # 마지막 row 셀 찾기
    max_row = sheet.max_row
    f = open('ChatBotData.csv', 'a', encoding='utf-8', newline='')
    wr = csv.writer(f)
    for r in range(2, max_row):
        # 질문과 답이 모두 있는것만 csv파일에 넣기
        if sheet.cell(row=r, column=13).value and sheet.cell(row=r+1, column=16).value:
            q = sheet.cell(row=r, column=13).value
            a = sheet.cell(row=r+1, column=16).value
            if len(q) < 25 and len(a) < 25:
                wr.writerow([q, a, 0])
    f.close()
    wb.close()


names = ['vehicle.xlsx', 'traffic.xlsx', 'waterworks.xlsx', 'passport.xlsx']
for name in names:
    wb = openpyxl.load_workbook(name)
    sheet = wb['Sheet']
    max_row = sheet.max_row
    f = open('ChatBotData.csv', 'a', encoding='utf-8', newline='')
    wr = csv.writer(f)
    for r in range(2, max_row):
        q = sheet.cell(row=r, column=5).value
        a = sheet.cell(row=r, column=6).value
        if q and a:
            if len(q) < 25 and len(a) < 25:
                wr.writerow([q, a, 0])
    f.close()
    wb.close()

# 데이터 분석
## wordcloud 만들기

In [ ]:
# 데이터 분석

import pandas as pd
from konlpy.tag import Okt
from tqdm import tqdm
from wordcloud import WordCloud
import matplotlib.pyplot as plt

data = pd.read_csv('./ChatBotData.csv', encoding='utf-8')

q_sentences = list(data['Q'])
a_sentences = list(data['A'])

# 토크나이즈
q_token_sentences = []
a_token_sentences = []

okt = Okt() # 객체 생성

for s in tqdm(q_sentences):
    for token, tag in okt.pos(s.replace(' ', '')):
        if tag == 'Noun':
            q_token_sentences.append(token)
            
for s in tqdm(a_sentences):
    for token, tag in okt.pos(s.replace(' ', '')):
        if tag == 'Noun':
            a_token_sentences.append(token)

# 하나의 문자열로 만들기
q_token_sentences = ' '.join(q_token_sentences)
a_token_sentences = ' '.join(a_token_sentences)

# # jupyter notebook 한글 폰트 
# from matplotlib import font_manager, rc
# font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
# rc('font', family=font_name)

# 질문관련 wordcloud
q_wordcloud = WordCloud('C:/Windows/Fonts/NanumGothic.ttf').generate(q_token_sentences)

plt.imshow(q_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

# 대답관련 wordcloud
a_wordcloud = WordCloud('C:/Windows/Fonts/NanumGothic.ttf').generate(a_token_sentences)

plt.imshow(a_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()


# Project
## data.py

In [ ]:
PAD = "<PAD>"
STD = "<SOS>"
END = "<END>"
UNK = "<UNK>"

PAD_INDEX = 0
STD_INDEX = 1
END_INDEX = 2
UNK_INDEX = 3

MARKER = [PAD, STD, END, UNK]

# Req 1-1-1. 데이터를 읽고 트레이닝 셋과 테스트 셋으로 분리

In [ ]:
# Req 1-1-1. 데이터를 읽고 트레이닝 셋과 테스트 셋으로 분리
def load_data(file):
    datas = pd.read_csv(DEFINES.data_path, header=0)
    q, a = list(datas['Q']), list(datas['A'])
    
    train_q, test_q, train_a, test_a = train_test_split(q, a, test_size=0.35, random_state=321)
    return train_q, train_a, test_q, test_a

In [ ]:
# train_q, train_a, test_q, test_a = load_data("./data_in/ChatBotData.csv")

# Req 1-1-2. 텍스트 데이터에 정규화를 사용하여 ([~.,!?\"':;)(]) 제거

In [1]:
def prepro_noise_canceling(sentence):
    RE_FILTER = re.compile("[.,!?\"':;~()]")
    sentence = re.sub(RE_FILTER, "", sentence)
    return sentence

# Req 1-1-3. 텍스트 데이터에 토크나이징

### 알고리즘 순서
    - 텍스트 데이터 prepro_noise_canceling() 함수 처리
    - 띄어쓰기 단위로 나누기
    - 띄어진 단어들로 벡터 형성

In [ ]:
def tokenizing_data(data):
    data = prepro_noise_canceling(data)
    data_splited = list(data.split())
    
    return data_splited

# Req 1-2-1. 토큰화된 트레이닝 데이터를 인코더에 활용할 수 있도록 전 처리

### 다음 내용들이 적용되어야 함
    - 텍스트 데이터 prepro_noise_canceling()함수 처리
    - 문장을 토큰 단위로 나누기
    - dictionary를 활용하여 토큰 인덱스화
    - dictionary에 없는 토큰의 경우 UNK 값으로 대체
    - 기준 문장 길이 보다 크게 된다면 뒤의 토큰 자르기
    - 기준 문장 길이에 맞게 남은 공간이 있다면 pedding 하기
    - 문장 인덱스와 문장 길이 계산

In [1]:
# Req 1-2-1. 토큰화된 트레이닝 데이터를 인코더에 활용할 수 있도록 전 처리
def enc_processing(value, dictionary):
    
    # 인덱스 정보를 저장할 배열 초기화
    seq_input_index = []
    # 문장의 길이를 저장할 배열 초기화
    seq_len = []
    # 노이즈 캔슬
    value = prepro_noise_canceling(value)
    
    for seq in value:
        # 하나의 seq에 index를 저장할 배열 초기화
        seq_index =[]

        for word in seq.split():
            if dictionary.get(word) is not None:
                # seq_index에 dictionary 안의 인덱스를 extend 한다
                seq_index.append(dictionary[word])
            else:
                # dictionary에 존재 하지 않는 다면 UNK 값을 extend 한다
                seq_index.append(dictionary[UNK])

        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 제거
        if len(seq_index) > DEFINES.max_sequence_length:
            seq_index = seq_index[:DEFINES.max_sequence_length]

        # seq의 길이를 저장
        seq_len.append(len(seq_index))

        # DEFINES.max_sequence_length 길이보다 작은 경우 PAD 값을 추가 (padding)
        seq_index += (DEFINES.max_sequence_length - len(seq_index)) * [dictionary[PAD]]

        # 인덱스화 되어 있는 값은 seq_input_index에 추가
        seq_input_index.append(seq_index)

    return np.asarray(seq_input_index)

# Req 1-2-2. 디코더에 필요한 데이터 전 처리 

### 다음 내용들이 적용되어야 함
    - 텍스트 데이터 prepro_noise_canceling()함수 처리
    - 문장을 토큰 단위로 나누기
    - dictionary를 활용하여 토큰 인덱스화
    - 첫 인덱스에 STD 추가
    - dictionary에 없는 토큰의 경우 UNK 값으로 대체
    - 기준 문장 길이 보다 크게 된다면 뒤의 토큰 자르기
    - 기준 문장 길이에 맞게 남은 공간이 있다면 pedding 하기
    - 문장 인덱스와 문장 길이 계산

In [1]:

def dec_input_processing(value, dictionary):
    
    # 인덱스 정보를 저장할 배열 초기화
    seq_input_index = []
    # 문장의 길이를 저장할 배열 초기화
    seq_len = []
    # 노이즈 캔슬
    value = prepro_noise_canceling(value)

    for seq in value:
        # 하나의 seq에 index를 저장할 배열 초기화
        seq_index =[]

        for word in seq.split():
            # 디코딩 입력의 처음에는 START가 와야 하므로 STD 값 추가
            seq_index.append(dictionary[STD])

            if dictionary.get(word) is not None:
                # seq_index에 dictionary 안의 인덱스를 extend 한다
                seq_index.append(dictionary[word])
            else:
                # dictionary에 존재 하지 않는 다면 seq_index에 UNK 값을 extend 한다
                seq_index.append(dictionary[UNK])

        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 제거
        if len(seq_index) > DEFINES.max_sequence_length:
            seq_index = seq_index[:DEFINES.max_sequence_length]

        # seq의 길이를 저장
        seq_len.append(len(seq_index))

        # DEFINES.max_sequence_length 길이보다 작은 경우 PAD 값을 추가 (padding)
        seq_index += (DEFINES.max_sequence_length - len(sequence_index)) * [dictionary[PAD]]

        # 인덱스화 되어 있는 값은 seq_input_index에 추가
        seq_input_index.append(seq_index)
    
    return np.asarray(seq_input_index)

# Req 1-2-3. 디코더에 필요한 데이터 전 처리 

In [1]:
# Req 1-2-3. 디코더에 필요한 데이터 전 처리 
def dec_target_processing(value, dictionary):
    
    # 인덱스 정보를 저장할 배열 초기화
    seq_input_index = []
    # 문장의 길이를 저장할 배열 초기화
    seq_len = []
    # 노이즈 캔슬
    value = prepro_noise_canceling(value)

    for seq in value:

        # 하나의 seq에 index를 저장할 배열 초기화
        seq_index =[]

        seq_index = [dictionary[word] for word in seq.split()]
        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 제거
        # END 토큰을 추가 (DEFINES.max_sequence_length 길이를 맞춰서 추가)
        
        seq_index = seq_index[:DEFINES.max_sequence_length-1] + [dictionary[END]]

        # seq의 길이를 저장
        seq_len.append(len(seq_index))

        # DEFINES.max_sequence_length 길이보다 작은 경우 PAD 값을 추가 (padding)
        seq_index += ( DEFINES.max_sequence_length - len(seq_index)) * [dictionary[PAD]]

        # 인덱스화 되어 있는 값은 seq_input_index에 추가
        seq_input_index.append(seq_index)
   
    return np.asarray(seq_input_index)